# Bounding configuration GP example

This notebook is designed to test whether a simple simple comprised of bounding boxes can be generated using [deap](http://deap.readthedocs.io/en/master/api/tools.html), a Python Evolutionary Algorithm Package.

In [1]:
import copy

import numpy as np

from functools import partial

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp


In [2]:
# def progn(*args):
#     for arg in args:
#         arg()

# def prog2(out1, out2): 
#     return partial(progn,out1,out2)

# def prog3(out1, out2, out3):     
#     return partial(progn,out1,out2,out3)

# def if_then_else(condition, out1, out2):
#     out1() if condition() else out2()



In [3]:
from OCC.gp import gp_Pnt
from airconics.base import AirconicsShape, AirconicsCollection
import OCC
from airconics import AirCONICStools as act
from OCC.BRepPrimAPI import BRepPrimAPI_MakeBox


# Create a simple Box class that inherits from AirconicsShape
#  - this will behave similarly to the Engine, Fuselage class etc.
class Box(AirconicsShape):
    def __init__(self, xmin, ymin, zmin, dx, dy, dz):
        # This implicitly calls build
        super(Box, self).__init__(construct_geometry=True,
                                             xmin=xmin,
                                             ymin=ymin,
                                             zmin=zmin,
                                             dx=dx,
                                             dy=dy,
                                             dz=dz)
        
        
        xmin, ymin, zmin, xmax, ymax, zmax = self.Extents()
        self.xmin = xmin
        self.ymin = ymin
        self.zmin = zmin
        self.xmax = xmax
        self.ymax = ymax
        self.zmax = zmax
        
        directions = []
        
    def Build(self):
        Xmin = gp_Pnt(self.xmin, self.ymin, self.zmin)
        Xmax = gp_Pnt(self.xmin + self.dx, self.ymin + self.dy, self.zmin + self.dz)
        self['Box'] = BRepPrimAPI_MakeBox(Xmin, Xmax).Shape()

        

# def progn(*args):
#     """LSIP progn connective"""
#     for arg in args:
#         arg()



# def fit2(out1, out2):
#     return partial(fitN, out1, out2)

# def fit2(out1, out2):
#     return partial(fitN, out1, out2)

        
class BoxLayout(AirconicsCollection):
    def __init__(self):
        """maxcompoents is the number of components"""
        super(BoxLayout, self).__init__(parts={}, construct_geometry=True)
                
        # Start with a simple box
        self.nparts = 0
        self.routine = None
    
    def run(self, routine):
        self._reset()
        routine()

    def _reset(self):
        self._Parts = {}
        self.nparts = 0

    def boxN(self, xmin, ymin, zmin, dx, dy, dz, *args):
        # Fits N new components to this box layout
        box = Box(xmin, ymin, zmin, dx, dy, dz)
        self.nparts += 1
        # Do no be confused between the numbering of boxes and the number of descendent
        #  nodes: Each box needs a unique ID, so naming a box0 function "box0" replaces
        #  other shapes in this layout that are also named box0
        self['box' + str(self.nparts)] = box
        for arg in args:
            arg()
    
    def box0(self, xmin, ymin, zmin, dx, dy, dz):
#         box = Box(xmin, ymin, zmin, dx, dy, dz)
#         self.nparts += 1
#         self['box0'] = box
        return partial(self.boxN, xmin, ymin, zmin, dx, dy, dz)
        
    # Now we need to explicitly add the box1, box2, .... boxM functions,
    # So that they can be added to the GP toolbox
    def box1(self, xmin, ymin, zmin, dx, dy, dz, out1):
#         box = Box(xmin, ymin, zmin, dx, dy, dz)
#         self.nparts += 1
#         self['box1'] = box
        return partial(self.boxN, xmin, ymin, zmin, dx, dy, dz, out1)

    def box2(self, xmin, ymin, zmin, dx, dy, dz, out1, out2):
#         box = Box(xmin, ymin, zmin, dx, dy, dz)
#         self.nparts += 1
#         self['box2'] = box
        return partial(self.boxN, xmin, ymin, zmin, dx, dy, dz, out1, out2)
    
    def temp_fitness(self):
        """Until I come with something useful, the fitness evaluation for this
        configuration will be based on the bounding box volume"""
        try:
            xmin, ymin, zmin, xmax, ymax, zmax = self.Extents()
        except:
            # Bounding Box was probably void
            return 0
        return (xmax-xmin) * (ymax-ymin)*(zmax-zmin)


In [4]:
config = BoxLayout()
import types, itertools, functools
funct = types.FunctionType
part = functools.partial
nt = types.NoneType

# I think the second argument means that we have up to 57 floats to use as terminals
pset = gp.PrimitiveSetTyped("MAIN", [], nt)


pset.addPrimitive(config.box0, [float, float, float, float, float, float], nt)
pset.addPrimitive(config.box1, [float, float, float, float, float, float, nt], nt)
pset.addPrimitive(config.box2, [float, float, float, float, float, float, nt, nt], nt)
pset.addPrimitive(np.random.rand, [], float)

def useless_None():
    return None
pset.addTerminal(useless_None, nt)

# terminals
# for i in range(100):
# label = 'rand' + str(i)
pset.addEphemeralConstant('rand', np.random.rand, float)
# for i in range(100):
#     pset.addTerminal(1.0, float)

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

# Attribute generator
toolbox.register("expr_init", gp.genFull, pset=pset, min_=1, max_=4)

# Structure initializers
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr_init)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


def evalTopology(individual):
    # Transform the tree expression to functionnal Python code
    routine = gp.compile(individual, pset)
    # Run the generated routine
    global config
    config.run(routine)
    return config.temp_fitness(),
    
toolbox.register("evaluate", evalTopology)
toolbox.register("select", tools.selTournament, tournsize=7)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

def main():
    np.random.seed(69)
    
    pop = toolbox.population(n=100)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)
    
    algorithms.eaSimple(pop, toolbox, 0.5, 0.2, 40, stats, halloffame=hof)
    
    return pop, hof, stats



Attempting to construct BoxLayout geometry...


In [5]:
tree = toolbox.individual()
print(tree)

routine = gp.compile(tree, pset)

config.run(routine)
print(config)


from airconics.Addons.WebServer.TornadoWeb import TornadoWebRenderer

renderer = TornadoWebRenderer()

config.Display(renderer)

renderer

box2(rand(), rand(), rand(), rand(), rand(), rand(), box0(rand(), rand(), rand(), rand(), rand(), rand()), box1(rand(), rand(), rand(), rand(), rand(), rand(), box1(0.8559478139330504, 0.23636939031326432, 0.09413675632557739, 0.21459347279717478, 0.4535901067611645, 0.9589239008900856, useless_None)))
['box4', 'box1', 'box3', 'box2']


<TornadoWebRenderer at http://PChambers_ngcm:44853/get/ec0c8211bf5b4c5abdbc32bde585c61b>

In [6]:
pop, hof, stats = main()

hof

gen	nevals	avg     	std     	min        	max   
0  	100   	0.776835	0.981554	0.000253284	4.7885
1  	56    	2.21246 	1.21495 	0.0130649  	5.02109
2  	55    	3.6524  	1.06848 	0.691369   	5.54205
3  	51    	4.61513 	0.666475	2.31551    	6.15174
4  	61    	4.76905 	0.836857	0.00939944 	6.15174
5  	49    	5.15842 	0.679957	3.0528     	6.19102
6  	67    	5.17459 	0.663725	3.98551    	6.19102
7  	50    	5.43424 	0.72989 	3.48645    	6.23375
8  	57    	5.315   	0.867995	3.55278    	6.23375
9  	49    	5.3912  	0.929574	2.76762    	6.23375
10 	54    	5.38691 	0.963746	3.14335    	6.6622 
11 	58    	5.29803 	0.932693	3.50208    	6.6622 
12 	64    	5.23758 	0.997455	0          	6.6622 
13 	58    	5.43519 	0.895513	3.52531    	6.6622 
14 	59    	5.38021 	1.02047 	3.18768    	6.6622 
15 	58    	5.50567 	1.03899 	3.50195    	6.6622 
16 	50    	5.66641 	1.03702 	3.62008    	6.6622 
17 	62    	5.42471 	1.0846  	3.35389    	6.6622 
18 	55    	5.55493 	1.09307 	3.35608    	6.6622 
19 	58    	5.43957 	1.

In [7]:
best = hof[0]

nodes, edges, labels = gp.graph(best)
print(best)

routine = gp.compile(best, pset)
config.run(routine)

print(config)

from airconics.Addons.WebServer.TornadoWeb import TornadoWebRenderer

renderer = TornadoWebRenderer()

config.Display(renderer)
renderer

box2(rand(), rand(), rand(), rand(), rand(), rand(), box2(rand(), rand(), rand(), rand(), rand(), rand(), box1(rand(), rand(), rand(), rand(), rand(), rand(), box1(0.9876911143597368, 0.5309143712841541, 0.6804361576402314, 0.39113473324146986, 0.11655619545171181, 0.70722572636939, useless_None)), box1(rand(), rand(), rand(), rand(), rand(), rand(), box1(0.4136160945599694, 0.4560330320768555, rand(), 0.0937152433162668, 0.4577166960483934, 0.8057929087049822, box2(rand(), rand(), rand(), rand(), rand(), rand(), box2(0.4136160945599694, rand(), rand(), rand(), rand(), rand(), box1(0.4468665688517708, 0.22445298556976356, 0.8856783021807887, 0.26203531051078377, 0.09049883039021211, 0.6738562352683545, useless_None), box2(0.04235634543141953, 0.33140249622173956, 0.4345371240025149, 0.8886938826657497, 0.43078742016811866, 0.724249867043646, useless_None, useless_None)), box0(rand(), rand(), rand(), rand(), 0.26203531051078377, rand()))))), box2(rand(), rand(), rand(), rand(), rand(), 

<TornadoWebRenderer at http://PChambers_ngcm:42432/get/a4de9ca05d2943c88f84cb4add8c6148>

In [8]:
config = BoxLayout(10)

def make_box(xmin, ymin, zmin, dx, dy, dz):
    def box0():
        global config
        config.box0(xmin, ymin, zmin, dx, dy, dz)
        return None
    return box0

p = config.box2(0,0,0,1,1,1, config.box0(1,1,1, 1, 1, 1), config.box0(2,2,2, 1, 1, 1))

res = p()

from airconics.Addons.WebServer.TornadoWeb import TornadoWebRenderer

renderer = TornadoWebRenderer()

print(config)
config.Display(renderer)

renderer

TypeError: __init__() takes exactly 1 argument (2 given)

In [ ]:
np.random.rand()

In [ ]:
help(gp.PrimitiveSetTyped)

In [ ]:
help(gp.PrimitiveSet)

In [ ]:
import itertools

r = [itertools.repeat(float, 57)] + [funct]

for entry in r:
    print(entry)

In [ ]:
from airconics.Addons.WebServer.TornadoWeb import TornadoWebRenderer

renderer = TornadoWebRenderer()
# b = Box(0,0,0,1,1,1)
box = BRepPrimAPI_MakeBox(gp_Pnt(0,0,0), gp_Pnt(1,1,1)).Shape()
renderer.DisplayShape(box)
renderer

In [ ]:
type(program)

In [ ]:
b = program()

In [ ]:
np.random.rand()

In [ ]:
from airconics.Addons.WebServer.TornadoWeb import TornadoWebRenderer

renderer = TornadoWebRenderer()
b.Display(renderer)
renderer